In [63]:
import pandas as pd
import numpy as np
import json

Se lee el archivo csv (incidencia delictiva) para hacer un Dataframe y lo mostramos (los casos aparecen por año y en la misma columna muestra todos los casos que se obtuvieron durante todo ese mismo año por tipo de delito, etc.)

In [34]:
df_delitos_2015_2020 = pd.read_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\Municipal-Delitos-2015-2022_mar2022.csv', sep=',', encoding='latin-1')
df_delitos_2015_2020.head()

,Año,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,...,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2,...,1,1.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,1.0
1,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,...,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,...,1,1.0,3.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
3,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,1,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0,...,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Se crea un diccionario donde vamos a transformar los datos, ya que para hacer el analisis posterior necesitamos los datos con fechas ej, "2015-01-31", por eso es necesario leer todo el archivo y hacer la transformación de los datos

In [8]:
datos_violencia = {
    "datos_violencia": []
}

for i in range(0,len(df_delitos_2015_2020)):
  mes = 0
  for j in range(9,21):
    mes = mes + 1
    nombre_fecha = f'{df_delitos_2015_2020.loc[i][0]}-{mes}-30'
    json_datos = {
      'fecha': nombre_fecha,
      'id_estado': df_delitos_2015_2020.loc[i][1],
      'estado': df_delitos_2015_2020.loc[i][2],
      'id_municipio':  df_delitos_2015_2020.loc[i][3],
      'municipio': df_delitos_2015_2020.loc[i][4],
      'bien_juridico': df_delitos_2015_2020.loc[i][5],
      'tipo_delito': df_delitos_2015_2020.loc[i][6],
      'sub_tipo_delito': df_delitos_2015_2020.loc[i][7],
      'modalidad': df_delitos_2015_2020.loc[i][8],
      'cantidad': df_delitos_2015_2020.loc[i][j],
    }
    datos_violencia["datos_violencia"].append(json_datos)


Obtenemos los datos en un diccionario ahora creamos un dataframe, guardamos el archivo en formato csv, ya que lo tenemos guardado en memoria y es necesario tener este punto almacenado en algun lugar

In [40]:
df = pd.DataFrame.from_dict(datos_violencia["datos_violencia"]
df.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\datos_violencia_fecha.csv')  

Leemos el archivo anteriormente guardado

In [5]:
df = pd.read_csv (r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\datos_violencia_fecha.csv')
df_incidencia_delictiva = df

Eliminamos la primera columna que se genero llamada 'Unnamed: 0'

In [7]:
df_incidencia_delictiva = df_incidencia_delictiva.drop(['Unnamed: 0'], axis=1)

Vemos informacion de los datos, que tipo de dato es cada columna, ya que vamos a cambiar algunos, como fecha

In [15]:
df_incidencia_delictiva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21988848 entries, 0 to 21988847
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   fecha            object 
 1   id_estado        int64  
 2   estado           object 
 3   id_municipio     int64  
 4   municipio        object 
 5   bien_juridico    object 
 6   tipo_delito      object 
 7   sub_tipo_delito  object 
 8   modalidad        object 
 9   cantidad         float64
dtypes: float64(1), int64(2), object(7)
memory usage: 1.6+ GB


Creamos un diccionario para cambiar el tipo de dato fecha

In [19]:
diccionario_de_conversion = {
    'fecha': 'datetime64[ns]'
}

Esta función va cambiar la fecha ya que anteriormente se agrego a todas las fechas el dia 30, y febrero no tiene día 30 por lo que tenemos que cambiarlo, aprovechamos a cambiar el día a 31 en los meses que cuentan con este día.

In [25]:
def change_date(data):
  for anio in range(2015,2023):
    if data == f'{anio}-1-30':
      return f'{anio}-1-31'
    elif data == f'{anio}-2-30':
      return f'{anio}-2-28'
    elif data == f'{anio}-3-30':
      return f'{anio}-3-31'
    elif data == f'{anio}-5-30':
      return f'{anio}-5-31'
    elif data == f'{anio}-7-30':
      return f'{anio}-7-31'
    elif data == f'{anio}-8-30':
      return f'{anio}-8-31'
    elif data == f'{anio}-10-30':
      return f'{anio}-10-31'
    elif data == f'{anio}-12-30':
      return f'{anio}-12-31'
  return data

Usamos la función map para cambiar la fecha, solo aplicamos este cambio a la columna fecha

In [26]:
df_incidencia_delictiva['fecha'] = df_incidencia_delictiva['fecha'].map(change_date)

In [27]:
df_incidencia_delictiva.head()

,fecha,id_estado,estado,id_municipio,municipio,bien_juridico,tipo_delito,sub_tipo_delito,modalidad,cantidad
0,2015-1-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2.0
1,2015-2-28,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0
2,2015-3-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0
3,2015-4-30,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0
4,2015-5-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0


Ahora si cambiamos el tipo de dato fecha al dataframe y lo mostramos y guardamos el archivo final en formato csv

In [28]:
temp = df_incidencia_delictiva.astype(diccionario_de_conversion)

temp.head()

,fecha,id_estado,estado,id_municipio,municipio,bien_juridico,tipo_delito,sub_tipo_delito,modalidad,cantidad
0,2015-01-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2.0
1,2015-02-28,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0
2,2015-03-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0
3,2015-04-30,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0
4,2015-05-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0


In [31]:
temp.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\datos_violencia_fecha_date_time.csv')  

Obtenemos los datos de la columna bien_juridico para hacer un dataframe de los unicos que existen y hacer una nueva tabla con esta información, despues guardamos la información en un csv

In [72]:
df_IDT = temp

ids_bien_juridico = []
data_bien_juridico = df_IDT['bien_juridico'].unique()
data_bien_juridico = data_bien_juridico.tolist()
data_bien_juridico.sort()

for id in range(1,len(data_bien_juridico)+1):
    ids_bien_juridico.append(id)
    
data_bien_juridico = pd.DataFrame({'id':ids_bien_juridico ,
                   'bien_juridico':data_bien_juridico})
data_bien_juridico.set_index('id', inplace = True)

data_bien_juridico.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\bien_juridico.csv',encoding='latin-1')  
data_bien_juridico

,bien_juridico
id,
1,El patrimonio
2,La familia
3,La libertad y la seguridad sexual
4,La sociedad
5,La vida y la Integridad corporal
6,Libertad personal
7,Otros bienes jurídicos afectados (del fuero co...


Hacemos lo mismo del paso anterior pero ahora para tipo de delito

In [74]:
ids_tipo_delito = []
data_tipo_delito = df_IDT['tipo_delito'].unique()
data_tipo_delito = data_tipo_delito.tolist()
data_tipo_delito.sort()

for id in range(1,len(data_tipo_delito)+1):
    ids_tipo_delito.append(id)
data_tipo_delito = pd.DataFrame({'id':ids_tipo_delito ,
                   'tipo_delito':data_tipo_delito})
data_tipo_delito.set_index('id', inplace = True)

data_tipo_delito.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\tipo_delito.csv',encoding='latin-1')  
data_tipo_delito

,tipo_delito
id,
1,Aborto
2,Abuso de confianza
3,Abuso sexual
4,Acoso sexual
5,Allanamiento de morada
6,Amenazas
7,Contra el medio ambiente
8,Corrupción de menores
9,Daño a la propiedad


Hacemos lo mismo del paso anterior pero ahora para sub tipo de delito

In [78]:
ids_sub_tipo_delito = []
data_sub_tipo_delito = df_IDT['sub_tipo_delito'].unique()
data_sub_tipo_delito = data_sub_tipo_delito.tolist()
data_sub_tipo_delito.sort()

for id in range(1,len(data_sub_tipo_delito)+1):
    ids_sub_tipo_delito.append(id)
data_sub_tipo_delito = pd.DataFrame({'id':ids_sub_tipo_delito ,
                   'sub_tipo_delito':data_sub_tipo_delito})
data_sub_tipo_delito.set_index('id', inplace = True)

data_sub_tipo_delito.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\sub_tipo_delito.csv',encoding='latin-1')  
data_sub_tipo_delito

,sub_tipo_delito
id,
1,Aborto
2,Abuso de confianza
3,Abuso sexual
4,Acoso sexual
5,Allanamiento de morada
6,Amenazas
7,Contra el medio ambiente
8,Corrupción de menores
9,Daño a la propiedad


Lo hacemos ahora para modalidad

In [77]:
ids_modalidad = []
data_modalidad = df_IDT['modalidad'].unique()
data_modalidad = data_modalidad.tolist()
data_modalidad.sort()

for id in range(1,len(data_modalidad)+1):
    ids_modalidad.append(id)
data_modalidad = pd.DataFrame({'id':ids_modalidad ,
                   'modalidad':data_modalidad})
data_modalidad.set_index('id', inplace = True)

data_modalidad.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\modalidad.csv',encoding='latin-1')  
data_modalidad

,modalidad
id,
1,Aborto
2,Abuso de confianza
3,Abuso sexual
4,Acoso sexual
5,Allanamiento de morada
6,Amenazas
7,Con arma blanca
8,Con arma de fuego
9,Con otro elemento


Se genera la data de estado y la guardamos en un archivo csv

In [219]:
ids_estado = df_IDT['id_estado'].unique()
data_estado = df_IDT['estado'].unique()

data_estado = pd.unique(pd.Index(df_IDT[['id_estado','estado']])).tolist()
data_estado = pd.DataFrame(data_estado)
data_estado.columns = ['id','estado']
data_estado.set_index('id', inplace = True)

data_estado.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\estados.csv',encoding='latin-1')  
data_estado

,estado
id,
1,Aguascalientes
2,Baja California
3,Baja California Sur
4,Campeche
5,Coahuila de Zaragoza
6,Colima
7,Chiapas
8,Chihuahua
9,Ciudad de México


Se genera la data de municipio y la guardamos en un archivo csv

In [220]:
ids_municipio = df_IDT['id_municipio'].unique()
data_municipio = df_IDT['municipio'].unique()

data_municipio = pd.unique(pd.Index(df_IDT[['id_municipio','municipio','id_estado']])).tolist()
data_municipio = pd.DataFrame(data_municipio)
data_municipio.columns = ['id','municipio','id_estado']
data_municipio.set_index('id', inplace = True)

data_municipio.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\municipios.csv',encoding='latin-1')  
data_municipio

,municipio,id_estado
id,,
1001,Aguascalientes,1
1002,Asientos,1
1003,Calvillo,1
1004,Cosío,1
1005,Jesús María,1
...,...,...
7125,Honduras de la Sierra,7
17034,Coatetelco,17
17035,Xoxocotla,17


Se genera una función para agregar el id respectivo en este caso de la columna "bien_juridico" buscando en su dataframe el valor del id "(dataframe.index[dataframe[f'{column}'] == value].tolist())" y despues buscarlo dentro de sus valores "dataframe[f'{column}'].index[pos[0]-1]"

In [141]:
df_IDT_temp = df_IDT

In [145]:
def get_id(value,dataframe,column):
    pos = (dataframe.index[dataframe[f'{column}'] == value].tolist())
    return dataframe[f'{column}'].index[pos[0]-1]

#df_IDT_temp["bien_juridico"].map(lambda data: get_id(data,data_bien_juridico,"bien_juridico"))
#df_IDT_temp['id_bien_juridico'] = df_IDT_temp['bien_juridico'].apply(lambda data: get_id(data,data_bien_juridico,"bien_juridico"))

df_IDT_temp['id_bien_juridico'] = df_IDT_temp['bien_juridico'].apply(get_id, args=(data_bien_juridico,"bien_juridico",))


In [147]:
df_IDT_temp['id_tipo_delito'] = df_IDT_temp['tipo_delito'].apply(get_id, args=(data_tipo_delito,"tipo_delito",))
df_IDT_temp.head()

,fecha,id_estado,estado,id_municipio,municipio,bien_juridico,tipo_delito,sub_tipo_delito,modalidad,cantidad,id_bien_juridico,id_tipo_delito
0,2015-01-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2.0,5,19
1,2015-02-28,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0,5,19
2,2015-03-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0,5,19
3,2015-04-30,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0,5,19
4,2015-05-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0,5,19


Hacemos lo mismo pero ahora para sub_tipo_delito

In [148]:
df_IDT_temp['id_sub_tipo_delito'] = df_IDT_temp['sub_tipo_delito'].apply(get_id, args=(data_sub_tipo_delito,"sub_tipo_delito",))

In [150]:
df_IDT_temp.head()

,fecha,id_estado,estado,id_municipio,municipio,bien_juridico,tipo_delito,sub_tipo_delito,modalidad,cantidad,id_bien_juridico,id_tipo_delito,id_sub_tipo_delito
0,2015-01-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2.0,5,19,20
1,2015-02-28,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0,5,19,20
2,2015-03-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0,5,19,20
3,2015-04-30,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0,5,19,20
4,2015-05-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0,5,19,20


Ahora paramodalidad

In [152]:
df_IDT_temp['id_modalidad'] = df_IDT_temp['modalidad'].apply(get_id, args=(data_modalidad,"modalidad",))
df_IDT_temp.head()

,fecha,id_estado,estado,id_municipio,municipio,bien_juridico,tipo_delito,sub_tipo_delito,modalidad,cantidad,id_bien_juridico,id_tipo_delito,id_sub_tipo_delito,id_modalidad
0,2015-01-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2.0,5,19,20,8
1,2015-02-28,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0,5,19,20,8
2,2015-03-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0,5,19,20,8
3,2015-04-30,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.0,5,19,20,8
4,2015-05-31,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0,5,19,20,8


Se reorganizan las columnas

In [156]:
df_IDT_temp = df_IDT_temp[['fecha','id_estado','estado','id_municipio','municipio','id_bien_juridico','bien_juridico','id_tipo_delito','tipo_delito','id_sub_tipo_delito','sub_tipo_delito','id_modalidad','modalidad','cantidad']]
df_IDT_temp.head()

,fecha,id_estado,estado,id_municipio,municipio,id_bien_juridico,bien_juridico,id_tipo_delito,tipo_delito,id_sub_tipo_delito,sub_tipo_delito,id_modalidad,modalidad,cantidad
0,2015-01-31,1,Aguascalientes,1001,Aguascalientes,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,2.0
1,2015-02-28,1,Aguascalientes,1001,Aguascalientes,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,0.0
2,2015-03-31,1,Aguascalientes,1001,Aguascalientes,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,1.0
3,2015-04-30,1,Aguascalientes,1001,Aguascalientes,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,1.0
4,2015-05-31,1,Aguascalientes,1001,Aguascalientes,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,0.0


Guardamos el archivo final en formato csv

In [159]:
df_IDT_temp.to_csv(r'C:\Users\isai_\Downloads\Municipal-Delitos-2015-2022_mar2022\Inicidencia-Delictiva\Incidencia_Delictiva_Full.csv',encoding='latin-1')  


Hacemos una filtración de los datos buscando el estado y el tipo de delito

In [223]:
filtro_estado = df_IDT_temp['estado'] == 'Guanajuato'
filtro_tipo_delito = df_IDT_temp['tipo_delito'] == 'Homicidio'
df_IDT_temp[filtro_estado & filtro_tipo_delito].head()

,fecha,id_estado,estado,id_municipio,municipio,id_bien_juridico,bien_juridico,id_tipo_delito,tipo_delito,id_sub_tipo_delito,sub_tipo_delito,id_modalidad,modalidad,cantidad
377496,2015-01-31,11,Guanajuato,11001,Abasolo,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,0.0
377497,2015-02-28,11,Guanajuato,11001,Abasolo,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,0.0
377498,2015-03-31,11,Guanajuato,11001,Abasolo,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,1.0
377499,2015-04-30,11,Guanajuato,11001,Abasolo,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,0.0
377500,2015-05-31,11,Guanajuato,11001,Abasolo,5,La vida y la Integridad corporal,19,Homicidio,20,Homicidio doloso,8,Con arma de fuego,1.0


Casos por estado y tipo de delito

In [227]:
casos_estado_delito = df_IDT_temp.groupby(['estado', 'tipo_delito'])['cantidad'].sum()
df_casos_estado_delito = pd.DataFrame(casos_estado_delito)
df_casos_estado_delito.head()

cantidad
estado         tipo_delito                     
Aguascalientes Aborto                      55.0
               Abuso de confianza        4504.0
               Abuso sexual                19.0
               Acoso sexual                 0.0
               Allanamiento de morada    2928.0

Casos por estado, tipo de delito y fecha

In [230]:
casos_estado_delito_fecha = df_IDT_temp.groupby(['fecha','estado', 'tipo_delito'])['cantidad'].sum()
df_casos_estado_delito_fecha = pd.DataFrame(casos_estado_delito_fecha)
df_casos_estado_delito_fecha.head(100)

cantidad
fecha      estado              tipo_delito                     
2015-01-31 Aguascalientes      Aborto                       0.0
                               Abuso de confianza          41.0
                               Abuso sexual                 6.0
                               Acoso sexual                 0.0
                               Allanamiento de morada      13.0
...                                                         ...
           Baja California Sur Falsificación                2.0
                               Feminicidio                  0.0
                               Fraude                      69.0
                               Homicidio                   20.0
                               Hostigamiento sexual         0.0

[100 rows x 1 columns]

Casos por tipo de delito

In [232]:
casos_delito = df_IDT_temp.groupby(['tipo_delito'])['cantidad'].sum()
df_casos_delito = pd.DataFrame(casos_delito)
df_casos_delito.sort_values('cantidad', ascending=False)

,cantidad
tipo_delito,
Robo,5082953.0
Lesiones,1450720.0
Violencia familiar,1371262.0
Otros delitos del Fuero Común,1301191.0
Daño a la propiedad,930577.0
Amenazas,681846.0
Fraude,517116.0
Narcomenudeo,418196.0
Homicidio,294800.0
